# Calibrating WFI Exposures with RomanCal 

***

## Kernel Information and Read-Only Status

To run this notebook, please select the "Roman Calibration" kernel at the top right of your window.

This notebook is read-only. You can run cells and make edits, but you must save changes to a different location. We recommend saving the notebook within your home directory, or to a new folder within your home (e.g. <span style="font-variant:small-caps;">file > save notebook as > my-nbs/nb.ipynb</span>). Note that a directory must exist before you attempt to add a notebook to it.

## Imports
 Libraries used
- *romancal* for running the processing pipeline
- *roman_datamodels* for opening Roman WFI ASDF files
- *asdf* for opening Roman WFI ASDF files
- *os* for checking if files exist
- *copy* for making copies of Python objects
- *astropy.coordinates* for working with celestial coordinates
- *s3fs* for streaming files from an S3 bucket

In [ ]:
import roman_datamodels as rdm
import asdf
from astropy.coordinates import SkyCoord
import copy
import romancal
from romancal.pipeline import ExposurePipeline
import os
import s3fs

## Introduction
The purpose of this notebook is to calibrate Level 1 (L1; uncalibrated ramp cube) data with the Roman WFI science calibration pipeline RomanCal (Python package name `romancal`) to produce Level 2 (L2; calibrated rate image) exposure level data. To learn more, please visit the [RDox pages on the Exposure Level Pipeline](https://roman-docs.stsci.edu/data-handbook-home/roman-stsci-data-pipelines/exposure-level-pipeline).

Details about the Roman data levels can be found in the RDox article [Data Levels and Products](https://roman-docs.stsci.edu/data-handbook-home/wfi-data-format/data-levels-and-products). A L1 file contains a single uncalibrated ramp in units of Data Numbers (DN).  L1 files are three-dimensional data cubes, one dimension for time and two dimensions for image coordinates, that are shaped as  arrays with (N resultants, 4096 image rows, 4096 image columns). A resultant contains either one read or the arithmetic mean of multiple reads of the WFI detectors. L2 WFI files are calibrated rate images in instrumental units of DN / second.  They are two-dimensional arrays shaped as (4088 image rows, 4088 image columns). Note the smaller image size of L2 files, which is due to the removal of the 4-pixel border of reference pixels around the image during pipeline processing.

***

## Tutorial Data
In this tutorial, we use L1 WFI data files simulated with `romanisim`. We will use as an example the output from running the [Roman I-Sim](../romanisim/romanisim.ipynb) tutorial notebook. If you did not run the simulation tutorial, then the files are also stored in the Nexus S3 bucket. For more information on how to access these data, see the [Data Discovery and Access](../data_discovery_and_access/data_discovery_and_access.ipynb) tutorial.

## Run romancal on L1 Data
To run `romancal` on the L1 data, there are two options:
1. You can use the exposure-level pipeline to run all steps (basic), or
2. You can run one or more individual steps (advanced).

### Basic Example: Full Pipeline

The input file for our example is a WFI L1 ASDF file. We first check to see if we already have the file saved on disk (if the Roman I-Sim tutorial was run), and if not then we stream the L1 file into memory (as a datamodel) from the Nexus S3 bucket:

In [ ]:
l1_file = 'r0003201001001001004_0001_wfi01_f106_uncal.asdf'

if os.path.exists(l1_file):
    dm_l1 = rdm.open(l1_file)
else:
    asdf_dir_uri = 's3://roman-sci-test-data-prod-summer-beta-test/'
    fs = s3fs.S3FileSystem()

    asdf_file_uri = asdf_dir_uri + f'AAS_WORKSHOP/{l1_file}'
    with fs.open(asdf_file_uri, 'rb') as f:
        af = asdf.open(f)
        dm_l1 = rdm.open(af).copy()

First, let's take a look at what kind of data we have using the `type()` function:

In [ ]:
type(dm_l1)

Reading the ASDF file with `roman_datamodels` returns a `ScienceRawModel` datamodel, which is the datamodel for L1 files. We can also use the `.info()` method on the datamodel to look at the file contents:

In [ ]:
dm_l1.info()

We can see that this L1 file was made with Roman I-Sim as it contains the "romanisim" block inside the file.

Next, let's look at a basic example of running the complete pipeline.

The `save_results` optional parameter will save the resulting L2 datamodel as a file on your Nexus storage. You can enable this by setting the value to `True`. In our example, we will keep the output calibrated L2 datamodel (as the variable `result`) in memory without saving it locally. We have explicitly set `save_results=False`, however this is also the default behavior.

In [ ]:
result = ExposurePipeline.call(dm_l1, save_results=False)

In [ ]:
type(result)

As we can see, the output from the Exposure Pipeline is an `ImageModel` object, which is the datamodel for L2 files.

If you look in your file browser in the directory where you are running this tutorial, you will also see two other files were created with similar names to our L1 file. These have names like `*_cat.asdf` and `*_segm.asdf`. These are Level 4 (L4; high-level extracted information) files corresponding to a single-band source catalog and a segmentation map, respectively.

**IMPORTANT NOTE:** At this time, L4 products are still being developed and validated, and we expect significant changes in their format and the algorithms used to generate them. These L4 products are automatically created by the source catalog step (a necessary input to the Gaia alignment). We do not recommend the use of these products until they are fully validated.

We can also pass optional parameters to individual pipeline steps using a dictionary called `steps`. Here we show how to skip the source catalog step and the step that aligns the image with the Gaia astrometric catalog (this is the TweakReg step, which is named after the software used to update the image World Coordinate System (WCS)). Other optional parameters may be similarly set for individual steps, and more information can be found in the [romancal documentation](https://roman-pipeline.readthedocs.io/en/latest/index.html).

In [ ]:
result = ExposurePipeline.call(dm_l1, save_results=False, steps={'source_catalog': {'skip': True}, 'tweakreg': {'skip': True}})

If you examine the end of the pipeline log messages, you will see that the source catalog and tweakreg steps were skipped as expected. Since we skipped the source catalog step, the L4 source catalog and segmentation maps were not regenerated. If we want to check on the status of a step, we can also check the metadata of the output datamodel:

In [ ]:
result.meta.cal_step.source_catalog

For reference, the pipeline steps in order are:

- `romancal.dq_init.dq_init_step`: Bad pixel masking and data quality initialization
- `romancal.saturation.SaturationStep`: Saturation flagging up-the-ramp
- `romancal.refpix.RefPixStep`: 1/f noise correction
- `romancal.linearity.LinearityStep`: Classic non-linearity correction
- `romancal.dark_current.DarkCurrentStep`: Dark current subtraction
- `romancal.ramp_fitting.ramp_fit_step`: Jump detection and fitting up-the-ramp
- `romancal.assign_wcs.AssignWcsStep`: Initialize the WCS with the pointing information
- `romancal.flatfield.FlatFieldStep`: Apply the flat field to the data
- `romancal.photom.PhotomStep`: Populate photometric calibration information
- `romancal.source_detection.SourceCatalog`: Run source detection on the image, perform point spread function (PSF) fitting photometry, and generate a source catalog
- `romancal.tweakreg.TweakRegStep`: Match sources to Gaia and update WCS information

The [Exposure Pipeline](https://roman-docs.stsci.edu/data-handbook-home/roman-stsci-data-pipelines/exposure-level-pipeline) article on RDox provides more information on the Exposure Pipeline steps.

Note that the ramp fitting step transforms the datamodel in memory. Therefore, steps following ramp fitting cannot be applied to a data model that has not undergone ramp fitting, and similarly, steps preceeding ramp fitting should not be applied to a  data model after this step.

Once we are satisfied with our datamodel, if we want to we can save it to disk with the `.save()` method:

In [ ]:
result.save('my_roman_l2_file.asdf')

If you look at the file browser in the directory you ran this tutorial, you should see a new file called "my_roman_l2_file.asdf." Note: you may need to wait a moment or manually refresh the file browser before the file appears.

### Advanced Example: Running Individual Pipeline Steps

Now, for a more advanced use case, let's update the WCS based on the pointing information. An example use case may be that we simulated a L1 file, calibrated it with the Exposure Pipeline, and now we want to try shifting the pointing information and creating a new WCS to test the Gaia alignment. After editing any of the `meta.wcsinfo` values that we want to change, we can generate a new WCS by running the individual AssignWcsStep on our L2 ASDF file.

In [ ]:
l2_file = 'r0003201001001001004_0001_wfi01_f106_cal.asdf'

if os.path.exists(l2_file):
    dm = rdm.open(f)
else:
    asdf_dir_uri = 's3://roman-sci-test-data-prod-summer-beta-test/'
    fs = s3fs.S3FileSystem()

    asdf_file_uri = asdf_dir_uri + f'AAS_WORKSHOP/{l2_file}'
    with fs.open(asdf_file_uri, 'rb') as f:
        af = asdf.open(f)
        dm = rdm.open(af)
        original_wcs = copy.deepcopy(dm.meta.wcs)

Let's take a quick look at the file we just opened:

In [ ]:
dm.info()

The WCS that is initially populate in L2 files before alignment with Gaia is based on the pointing information. This is called the "course" WCS. The `meta.pointing` section of the metadata describes the spacecraft pointing, while the detector-dependent information used to construct the course WCS is contained in the `meta.wcsinfo` section. Realistically, the values in `meta.pointing` and `meta.wcsinfo` are linked, but in practice the course WCS only uses `meta.wcsinfo`. Let's examine our `meta.wcsinfo` values:

In [ ]:
dm.meta.wcsinfo

In particular, we see values for `ra_ref`, `dec_ref`, and `roll_ref`. Let's take a look at the descriptions of these fields:

In [ ]:
print(f"ra_ref = {dm.schema_info(path='roman.meta.wcsinfo.ra_ref')['description']}")
print(f"dec_ref = {dm.schema_info(path='roman.meta.wcsinfo.dec_ref')['description']}")
print(f"roll_ref = {dm.schema_info(path='roman.meta.wcsinfo.roll_ref')['description']}")

The "reference pixel position" in the descriptions is located at the center of each WFI detector (each detector has its own WCS). We can edit these values to trick the pipeline into creating a WCS slightly different from the one before. Let's make a copy of the datamodel (for comparison later)  and add a simple shift of 1 arcseccond in right ascension:

In [ ]:
original_ra_ref = copy.copy(dm.meta.wcsinfo.ra_ref)
dm.meta.wcsinfo.ra_ref += (1 / 3600)

print(f'Original ra_ref = {original_ra_ref},\nUpdated ra_ref = {dm.meta.wcsinfo.ra_ref}')

Next, let's run AssignWcsStep on the datamodel. Doing so will return an updated datamodel in memory:

In [ ]:
result = romancal.assign_wcs.AssignWcsStep.call(dm)

Finally, we can take a pixel position and translate it to right ascension and declination using the original and newly updated WCS objects. Let's use the center of the detector in the L2 image, which in 0-indexed pixels is (x, y) = (2043.5, 2043.5). Separations on the sky can be easily determined using `astropy.coordinates.SkyCoord` objects as follows:

In [ ]:
# Get SkyCoord object for new position at center of detector
ra, dec = result.meta.wcs(2043.5, 2043.5)
result_coord = SkyCoord(ra=ra, dec=dec, unit='deg')
result_coord

# Get SkyCoord object for original position at center of detector
ra0, dec0 = original_wcs(2043.5, 2043.5)
original_coord = SkyCoord(ra=ra0, dec=dec0, unit='deg')
original_coord

# Compute the separation between the updated and original positions
result_coord.separation(original_coord)

As we can see, the newly updated WCS is shifted approximately 1 arcsecond from the WCS in the L2 file that we opened. This is not exactly 1 arcsecond because the WFI pixels were not aligned with perfectly vertical and horizontal lines of right ascension and declination, respectively.

Similar to our pipeline example above, we can also pass optional arguments to individual steps. For example, if we would like to use our own version of the distortion reference file rather than the one from CRDS, then we can use the file with the override_distortion optional parameter:

In [ ]:
# wcs_step = romancal.assign_wcs.AssignWcsStep.call(dm, override_distortion='my_distortion_file.asdf')

Similar override parameters exist for all reference file types. More information on WFI reference file types may be found in the RDox article [CRDS for Reference Files](https://roman-docs.stsci.edu/data-handbook-home/accessing-wfi-data/crds-for-reference-files).

As before, we directed the updated datamodel to a variable in active memory. We can also pass this datamodel along to the next pipeline step and chain steps together, and we can also save the datamodel to disk with the `.save()` method. For more information on working with datamodels, see the [Working with ASDF](../working_with_asdf/working_with_asdf.ipynb) tutorial.

## Additional Resources
- [romanisim](https://romanisim.readthedocs.io/en/latest/index.html)
- [romancal](https://roman-pipeline.readthedocs.io/en/latest/index.html)
- [Roman Documentation](https://roman-docs.stsci.edu)

## About this Notebook
**Author:** Tyler Desjardins, Sanjib Sharma\
**Updated On:** 2025-05-26

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 